In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095452 sha256=420d77ea8f87a423a180f170c6314829aecc2826b69a1b1cd63260035b40f4f8
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 62.9 MB/s eta 0:00:00


In [4]:
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.lmf import LogisticMatrixFactorization

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

In [5]:
random_state_value = 42

id_column_name = 'cookie_id'

**Загружаем датасет**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data_dir = '/content/drive/MyDrive/хакатон_работа_ру'

In [8]:
train_df = pd.read_parquet(f"{data_dir}/Processed_dataset1.parquet")
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3


In [9]:
test_df = pd.read_parquet(f"{data_dir}/test_public_mfti.parquet")
test_df  

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [75]:
test_private_df = pd.read_parquet(f"{data_dir}/test_private_users_mfti.parquet")
      

In [76]:
test_private_df= test_private_df.assign(vacancy_id_=pd.Series([''] * len(test_private_df), dtype=object).values)   
test_private_df

,cookie_id,vacancy_id_
0,0018914ba3e54011b28fa715583d3354,
1,0035c298d8c64f368ae730a9cca9bb20,
2,00956458877448ec9fba87fb97443fdf,
3,0099387c921b41e7bae6c99dd8254b60,
4,009f65e8ae99413a8da94a491320580a,
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,
3082,ffbc08b528c64f22996873fc63872202,
3083,ffdeaf3c34544529880aebf17c103f6c,
3084,ffefa79a74804ee69e6c131e0d05b948,


In [13]:
nonrel_act_list = ['show_vacancy',
                'preview_click_vacancy',
                'click_favorite',
                'preview_click_favorite']

fav_act_list = ['click_favorite',
                'preview_click_favorite']

rel_act_list = ['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

In [70]:
submission_df = pd.read_parquet(f"{data_dir}/test_private_sample_submission_mfti.parquet")
submission_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [14]:
# Создаем колонку с типом действия для рейтинга
# 1 - целевое действие, 0.5 - добавление в избранное, 0 - остальные действия

rating_fav_column_name = 'rating_fav_05'
rat_add = 0.5

def check_is_purpose_2(event, rat_add):
    if event in rel_act_list:
        return 1
    elif event in fav_act_list:
        return rat_add
    else:
        return 0
    
train_df[rating_fav_column_name] = train_df['event_type'].apply(check_is_purpose_2, rat_add=rat_add)
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating_fav_05
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
...,...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0


In [ ]:
#удаляем аномальный cookie:
df_temp = pd.DataFrame(train_df.groupby(['vacancy_id_'])['cookie_id'].nunique().sort_values(ascending=False))
df_temp = pd.DataFrame(df_temp).reset_index()
suspicious_vacancy_lst = df_temp[df_temp['cookie_id']==1]['vacancy_id_'].tolist()
suspicious_vacancy_lst

In [18]:
suspicious_vacancy_lst = df_temp[df_temp['cookie_id']==1]['vacancy_id_'].tolist()
train_df = train_df[train_df.vacancy_id_.isin(suspicious_vacancy_lst) == False]
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating_fav_05
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
...,...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0


In [29]:
#удаляем первые 8 активных куки
#drop_cookie_pop = train_df.groupby('cookie_id')['vacancy_id_'].nunique().sort_values().tail(8).index.tolist()
#drop_cookie_pop

['9e058f6145b348d09da9aef70382582c',
 'fb1c48ea68c24cb28f7aee5c655dbfe8',
 '94f2efb02a214ef0bd310fcd53a7d98c',
 '7d8cc5a0fef44378a2d90a237cda288e',
 '6b3281f474314f96b75a7d1a06f09eed',
 '728c61d1db294cb59e6032a65706b964',
 '784a284e0bd34228903d95cff1559ca4',
 '924398d361a0454c8c30845c2b4c5747']

In [30]:
#train_df = train_df[~train_df['cookie_id'].isin(drop_cookie_pop)]
#train_df.shape

(12009304, 10)

In [50]:
'''
# Выбрать только те cookie_id, у которых количество вакансий больше 9 (1000 cлуйчаных) 
 
import random
random.seed(13)
# Выбрать cookie_id с количеством vacancy_id_ > 5
cookie_ids_to_exclude = train_df.groupby('cookie_id').filter(lambda x: (x['vacancy_id_'].nunique() > 5) )['cookie_id'].unique()

# Выбрать случайные 1000 уникальных cookie_id
cookie_ids_to_exclude = random.sample(list(cookie_ids_to_exclude), 1000)

# Выбрать только те cookie_id, которые нужно исключить
train_5_exclude = train_df[train_df['cookie_id'].isin(cookie_ids_to_exclude)]

# Получить последние 5 vacancy_id_ с рейтингом == 1 для каждого cookie_id исключаемых из train_5
test_5 = train_5_exclude.groupby('cookie_id').apply(lambda x: x[x['rating_fav_05'] > 0].tail(5)).reset_index(drop=True)
'''

In [51]:
#test_5 = test_5.rename(columns={'removed_vacancies':'vacancy_id_'})

In [52]:
#test_6 = test_5.groupby(['cookie_id'], as_index=False).agg({'vacancy_id_': list})
#test_6

,cookie_id,vacancy_id_
0,00df2fdac2f544e1a262a093fcd76be8,"[144917, 149530, 147729, 144909, 158238]"
1,014c81ff0d4a46d5bd68ee28026f9bcd,[249819]
2,015bbd72dd6f4a849c8d8d1f9611350d,[254027]
3,022e352c3f114262bd3f8b47042d8f04,"[174953, 174953, 116823, 114583, 188385]"
4,027598de4f174013ab6e90082a05e104,"[117400, 117400, 115174, 115174]"
...,...,...
647,fe6f03af965e4882a5328eadc238657b,"[242715, 183152, 242715, 242715, 242715]"
648,fec337c6af9e40ecaf100a5da26e241c,"[184363, 184363, 104978, 221474, 255744]"
649,fed4f13eb4194af092fbcf3a2ad801cb,"[113816, 243141, 205569, 110917]"
650,ff6eb5f2729045cdb164f353ec0f9093,[192554]


In [53]:
'''
# Извлечение vacancy_id из test_6 для каждого cookie_id
test_vacancies = test_6.set_index('cookie_id')['vacancy_id_'].to_dict()

# Удаление строк, соответствующих vacancy_id из test_6
train_5 = train_df[~train_df.apply(lambda x: x['vacancy_id_'] in test_vacancies.get(x['cookie_id'], []), axis=1)]
train_5
'''

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating_fav_05
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0.0
...,...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3,0.0
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1.0


**Для рейтинга 0-6 и с прибавкой rat_add=0.5 за добавление в избранное**

In [19]:
train_df_short = train_df[[id_column_name, 'vacancy_id_', 'event_type', rating_fav_column_name]]
train_matrix_rating0_6_fav = train_df_short.drop_duplicates(keep='first', ignore_index=True).sort_values(by=[id_column_name, 'vacancy_id_'], ignore_index=True)
train_matrix_rating0_6_fav = train_matrix_rating0_6_fav.groupby(by=[id_column_name, 'vacancy_id_'], as_index=False).agg({rating_fav_column_name: 'sum'})
train_matrix_rating0_6_fav

,cookie_id,vacancy_id_,rating_fav_05
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0.0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0.0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0.5
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0.0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0.0
...,...,...,...
4657104,ffffdb17f45b4032b386d691d52e6c00,107125,1.0
4657105,ffffdb17f45b4032b386d691d52e6c00,107893,1.0
4657106,ffffdb17f45b4032b386d691d52e6c00,150843,1.0
4657107,ffffdb17f45b4032b386d691d52e6c00,160164,1.0


In [20]:
f'min rating = {train_matrix_rating0_6_fav[rating_fav_column_name].min()}, max rating = {train_matrix_rating0_6_fav[rating_fav_column_name].max()}'

'min rating = 0.0, max rating = 7.0'

## Часть Implicit

In [21]:
col_names_fav ={
    'id_col_name': id_column_name,
    'vac_col_name': 'vacancy_id_',
    'rating_col_name': rating_fav_column_name
}

In [22]:
def transform_into_sparse_matrix(input_df, col_names):
    input_df_fixed = input_df.copy()
    input_df_fixed[col_names['id_col_name']] = input_df_fixed[col_names['id_col_name']].astype('category')
    input_df_fixed[col_names['vac_col_name']] = input_df_fixed[col_names['vac_col_name']].astype('category')
    input_df_fixed[col_names['rating_col_name']] = input_df_fixed[col_names['rating_col_name']].astype('uint8')
    user_item_matrix = csr_matrix((input_df_fixed[col_names['rating_col_name']], 
                                (input_df_fixed[col_names['id_col_name']].cat.codes, 
                                 input_df_fixed[col_names['vac_col_name']].cat.codes)))
    id_cat_codes = dict(zip(input_df_fixed[col_names['id_col_name']], input_df_fixed[col_names['id_col_name']].cat.codes))
    vac_cat_codes = dict(zip(input_df_fixed[col_names['vac_col_name']], input_df_fixed[col_names['vac_col_name']].cat.codes))
    cat_vac_codes = dict(zip(input_df_fixed[col_names['vac_col_name']].cat.codes, input_df_fixed[col_names['vac_col_name']]))
    return (user_item_matrix, id_cat_codes, vac_cat_codes, cat_vac_codes)


In [23]:
Impl_rat06_fav_set = transform_into_sparse_matrix(train_matrix_rating0_6_fav, col_names_fav)

In [25]:
Impl_ALS_rat06_fav = AlternatingLeastSquares(factors=50,
                                             regularization=0.05,
                                             calculate_training_loss=True,
                                             random_state=random_state_value) # , alpha=3.0
Impl_ALS_rat06_fav.fit(Impl_rat06_fav_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
# Создание списка неактульных вакансий, с которомы давно не было взаимодействий
# 78 973 вакансии не были провзаимодействованы за последние 2 недели рассматриваемого периода (prec = 0.031)
# 99 046 вакансии не были провзаимодействованы за последнюю неделю (prec = 0.033)
# 117 944 вакансии не были провзаимодействованы за последние 3 дня (prec = 0.036)
# 127 405 вакансии не были провзаимодействованы за последние 2 дня (prec = 0.038)
# 140 242 вакансии не были провзаимодействованы за последний 1 день - 0.04
non_valid_vacs = []
for vac, group in train_df.groupby('vacancy_id_'):
    max_date = group['event_date'].max()
    if max_date <= '2022-09-30':
        non_valid_vacs.append(vac)
len(non_valid_vacs)

118777

In [27]:
valid_vacs = train_matrix_rating0_6_fav[~train_matrix_rating0_6_fav['vacancy_id_'].isin(non_valid_vacs)]['vacancy_id_'].unique()
len(valid_vacs)

19914

In [78]:
# Задаем словари с моделями и датасетами чтобы перебрать их все

models_dict_Impl = {
    'Impl_ALS_rat06_fav_0.5': [Impl_ALS_rat06_fav, 'implicit', Impl_rat06_fav_set]
}

set_dict_Impl = {
    'non_val_vacs': valid_vacs
}

In [29]:
# Вспомогательные функции

def get_predictions(cookie_id, model_info, initial_set, n_predict = 5):
    # Получим все вакансии с которыми взаимодействовал cookie_id:
    interacted_items = train_matrix_rating0_6_fav[train_matrix_rating0_6_fav['cookie_id'] == cookie_id]['vacancy_id_'].unique()
    
    # Удалим из мешка вакансии с которыми контактировал юзер:
    items_to_predict = np.setdiff1d(initial_set, interacted_items)
    
    model = model_info[0]
    model_type = model_info[1]  
    
    # получим топ-n вакансий для данного юзера:
    if model_type == 'surprise':
        top_n_vacancies = get_predictions_surprise(cookie_id, model, items_to_predict, n_predict)
    
    if model_type == 'implicit':
        filtered_set = train_matrix_rating0_6_fav[~train_matrix_rating0_6_fav['vacancy_id_'].isin(items_to_predict)]['vacancy_id_'].unique()
        top_n_vacancies = get_predictions_implicit(cookie_id, model, filtered_set, model_info[2], n_predict)
    
    return top_n_vacancies


def get_predictions_surprise(user_id, model, vac_set, n_predict = 5):
    # Создадим тест датасет для данного юзера:
    test_set = [[user_id, item, 4.] for item in vac_set]        
    
    predictions = model.test(test_set)
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_predict]
    return [pred.iid for pred in top_n]


def get_predictions_implicit(user_id, model, filtered_set, impl_set, n_predict = 5):
    user_item = impl_set[0]
    id_cat_code = impl_set[1][user_id]
    vac_cat_codes = impl_set[2]
    cat_vac_codes = impl_set[3]
    filter_items = np.array([vac_cat_codes[i] for i in filtered_set])
    ids, scores = model.recommend(id_cat_code,
                                  user_item[id_cat_code],
                                  N=n_predict,
                                  filter_items=filter_items,
                                  filter_already_liked_items=True)
    return [cat_vac_codes[i] for i in ids]    


def calc_correct_recomend_number(predict_col, vac_col):
    """Функция для определения количества совпадающих вакансий в двух списках"""
    return len(set(predict_col) & set(vac_col))


def get_prec_n(res_df, num_corr_column, n_predict = 5):
    """Функция для расчета precision@n метрики"""
    corr_recommend = res_df[num_corr_column].sum()
    prec_5 = corr_recommend / (n_predict * res_df.shape[0])
    return {'num_corr_recomend': corr_recommend, f'precision@{n_predict}': prec_5}

In [79]:
# Копируем датасет чтобы не менять изначальный

test_df_with_pred = test_private_df.copy()

In [80]:
# Импортируем tqdm для progress bar
from tqdm.auto import tqdm

def get_res_df(models_dict, set_dict, n_predict = 5):
    res_dict = {}
    for model_name in tqdm(models_dict):        
        for set_name in tqdm(set_dict):        
            # Создаем столбец с предсказаниями
            predict_col_name = f'predict_id_{model_name}_{set_name}_n_{n_pred}'            
            test_df_with_pred[predict_col_name] = test_df_with_pred['cookie_id'].apply(get_predictions,
                                                                                       model_info = models_dict[model_name],
                                                                                       initial_set = set_dict[set_name],
                                                                                       n_predict = n_pred)                  
            
            # Создаем столбец с количеством правильных предсказаний для каждого id
            num_corr_col_name = f'num_corr_{model_name}_{set_name}_n_{n_pred}'        
            test_df_with_pred[num_corr_col_name] = test_df_with_pred.apply(
                lambda x: calc_correct_recomend_number(x[predict_col_name], x['vacancy_id_']), axis=1
            )
            
            # Считаем precision@n метрику
            res_dict[model_name+'_'+set_name] = get_prec_n(test_df_with_pred,
                                                           num_corr_column = num_corr_col_name,
                                                           n_predict = n_pred)
    return pd.DataFrame.from_dict(res_dict, orient='index')

In [81]:
n_pred = 5

res_df = get_res_df(models_dict_Impl, set_dict_Impl, n_predict = n_pred)
res_df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,num_corr_recomend,precision@5
Impl_ALS_rat06_fav_0.5_non_val_vacs,0,0.0


In [ ]:
pd.set_option('display.max_rows', None)
res_df.sort_values(by=['precision@5'], ascending=False)

,num_corr_recomend,precision@5
Impl_ALS_rat06_fav_0.5_non_val_vacs,151,0.039119
Impl_ALS_rat06_non_val_vacs,146,0.037824
Impl_ALS_rat06_fav_0.1_non_val_vacs,146,0.037824
Impl_ALS_rat06_fav_0.15_non_val_vacs,146,0.037824
Impl_ALS_rat06_fav_0.05_non_val_vacs,146,0.037824


In [ ]:
test_df_with_pred

In [88]:
'''
def get_top_n_vacancies(rating_act_list, pos_vacancies_start=0, pos_vacancies_end=100):
    show_act = train_df[train_df['event_type'].isin(rating_act_list)].groupby('vacancy_id_')['event_type'].count()
    top_n_with_event_quant = show_act.sort_values(ascending=False).iloc[pos_vacancies_start:pos_vacancies_end].to_frame().reset_index()
    return top_n_with_event_quant['vacancy_id_'].unique()
'''

In [83]:
result = test_df_with_pred[['cookie_id', 'predict_id_Impl_ALS_rat06_fav_0.5_non_val_vacs_n_5']].copy()
result = result.rename(columns = {'predict_id_Impl_ALS_rat06_fav_0.5_non_val_vacs_n_5': 'predictions'})
result

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[138634, 209422, 153970, 117525, 209448]"
1,0035c298d8c64f368ae730a9cca9bb20,"[193331, 207156, 212325, 149024, 113482]"
2,00956458877448ec9fba87fb97443fdf,"[116900, 217683, 101462, 259050, 122614]"
3,0099387c921b41e7bae6c99dd8254b60,"[103881, 246509, 153970, 149484, 144097]"
4,009f65e8ae99413a8da94a491320580a,"[138634, 181745, 153970, 154411, 193331]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[207423, 217683, 105907, 108242, 113305]"
3082,ffbc08b528c64f22996873fc63872202,"[181745, 115977, 154411, 247299, 234115]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[230707, 240744, 115977, 207423, 237341]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[220718, 253678, 122614, 101462, 246509]"


In [84]:
result.to_csv('result.csv', index=False)